<p style="color:#4169E1; font-family:'Roboto'; font-size:48px; text-align: center;"> 
Process WLS data
</p>

**Table of contents**<a id='toc0_'></a>    
- 1. [Change diary:](#toc1_)    
- 2. [Reading pickled event catalogue](#toc2_)    
- 3. [initialize Pyreco](#toc3_)    
  - 3.1. [Initialize manager](#toc3_1_)    
  - 3.2. [Event information](#toc3_2_)    
- 4. [Analysis](#toc4_)    
  - 4.1. [data selection](#toc4_1_)    
  - 4.2. [Plotting data](#toc4_2_)    
  - 4.3. [histogramming pretrigger sum values](#toc4_3_)    
  - 4.4. [Repetitive pattern seen in Ch-2 data](#toc4_4_)    
    - 4.4.1. [Fourier Transform](#toc4_4_1_)    
  - 4.5. [simultaneity of pulses](#toc4_5_)    
  - 4.6. [Center of Mass for waveforms](#toc4_6_)    
  - 4.7. [full waveform sum](#toc4_7_)    
- 5. [Exploration](#toc5_)    
- 6. [Memory diagnostics](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<p>
<b> Steps to follow: </b>

<ol>
<li> Event selection cuts </li>
<li> Histogram event charge</li>
<li> SPE calibration </li>
<li> cut efficiency: waveform integration post cut / pre cut bin by bin</li>
<li> Light Yield calculation</li>
<!-- <li> </li> -->
</ol>
</p>


<!-- # MIDAS events reconstruction -->

Load packages, setup libraries

In [ ]:
from time import perf_counter

t0 = perf_counter()

In [ ]:
import sys
import numpy as np
# from scipy.optimize import curve_fit
# from scipy.stats import chisquare
# from scipy.integrate import simpson

# import matplotlib
import matplotlib.pyplot as plt
# from matplotlib.ticker import AutoMinorLocator
# from matplotlib.ticker import FormatStrFormatter

# from termcolor import colored
# import latexify
import pandas as pd
# from tqdm.notebook import tqdm
import pickle

In [ ]:
from scipy.signal import find_peaks

In [ ]:
# from pyreco.manager.manager import Manager

In [ ]:
np.set_printoptions(formatter={'float': lambda x: f"{x:10.4g}"})

In [ ]:
%matplotlib ipympl

# 1. <a id='toc1_'></a>[Change diary:](#toc0_)
1. Marcin suggested to use these settings in pyreco for some runs:
`n32samples = 9168-1`
2. new pyreco branch: zle_4ksamp_length
3. Marcin suggested 3 WLS specific cuts
4. LY calculation w/o SPE calibration
5. LY calculation w/ SPE calibration

# 2. <a id='toc2_'></a>[Reading pickled event catalogue](#toc0_)

In [ ]:
# event_catalogue_file = f'../temp_folder/copy_event_catalogue_run0061.pkl'
# event_catalogue_file = f'../temp_folder/event_catalogue_run0061.pkl'
# event_catalogue_file = f'../data/event_catalogue_run00061_3ch.pkl'
event_catalogue_file = f'/work/chuck/sarthak/argset/event_catalogues/event_catalogue_run00108_first_half.pkl'
# event_catalogue_file = f'/work/chuck/sarthak/argset/event_catalogues/event_catalogue_run00152_00.pkl'
event_catalogue = pd.read_pickle(event_catalogue_file)

In [ ]:
wfs = event_catalogue['wf']

In [ ]:
# event_catalogue.info()

In [ ]:
# event_catalogue.shape

In [ ]:
# event_catalogue.head()

In [ ]:
# Plotting events
plt.figure()
# for event_id in range(9900, 9905):
for event_id in range(5000, 5001):
    for ch in range(3):
    # for ch in [2]:
        plt.plot(wfs[event_id][ch])

In [ ]:
# --> 1&2 waveforms shouldn't have maximum shifted by more than ~ 20 ns

# 3. <a id='toc3_'></a>[initialize Pyreco](#toc0_)

Select here midas data, output filename and configuration file

In [ ]:
# # filename = '/work/sarthak/argset/data/2024_Mar_27/midas/run00061.mid.lz4'
# filename = '/work/sarthak/argset/data/run00108.mid.lz4'
# # filename = '/work/sarthak/argset/data/run00124.mid.lz4'
# outfile = 'jupyR00108'
# confile = 'argset.ini'
# # tmin,tmax = 0, 1750

## 3.1. <a id='toc3_1_'></a>[Initialize manager](#toc0_)

In [ ]:
# cmdline_args = f'--config {confile} -o {outfile} -i {filename}'
# m = Manager( midas=True, cmdline_args=cmdline_args)
# # m = Manager( midas=True) #, cmdline_args=cmdline_args)
# # baseline_samples = m.config('reco', 'bl_to', 'int')

## 3.2. <a id='toc3_2_'></a>[Event information](#toc0_)

In [ ]:
# # nev_max = m.config('base', 'nevents', 'int')
# # nev_max = 2
# # nev_max = 455 # diag
# # for nev, event in enumerate(m.midas): # oddly doing m.midas shifts the 'pointer' to the next event
# for nev, event in enumerate(m.midas):
#     #print(nev, event, event.nchannels)
#     if nev < 2: continue
#     if nev > 1: break
#     # if nev > 450: break
#     # print(nev)

In [ ]:
# event.adc_baseline

In [ ]:
# # to use this cell, run the above cell once
# plt.close(1)
# plt.figure(1)
# #### Note: this way of looping skips one event every time we execute the for loop. Probably better not to use this loop in full code. 
# for nev, event in enumerate(m.midas): # oddly doing m.midas shifts the 'pointer' to the next event
#     if nev > 0: break
#     wfs = event.adc_data
#     print(event.event_counter)
#     for i,wf in enumerate(event.adc_data): # over channels
#         if event.event_counter:
#             wfs[i] = wf-event.adc_baseline[i]
#             print(event.event_counter)
#             plt.plot(wfs[i], label=f'{i}')
# # plt.xlim(tmin,tmax)
# # plt.legend()

In [ ]:
# del wf

In [ ]:
# event.adc_data

In [ ]:
# event.adc_baseline

In [ ]:
# wf

In [ ]:
# wfs[2]

In [ ]:
# np.sum(wfs[2][:400])

In [ ]:
# np.sum(wf[:400])

In [ ]:
# for nev, event in enumerate(m.midas): # oddly doing m.midas shifts the 'pointer' to the next event
#     if nev > 0: break
#     wfs = event.adc_data
#     if len(event.adc_data) != 0:
#         plt.figure()
#         for i,wf in enumerate(event.adc_data): # over channels
#             wfs[i] = wf-event.adc_baseline[i]
#             plt.plot(wfs[i], label=f'{i}')
#             # plt.xlim(tmin,tmax)
#             plt.legend()

In [ ]:
# event.event_counter
# event.adc_data.shape
# event.adc_data = np.array([])
# len(event.adc_data)
# if not event.adc_data:
#     print('Yes')
# event.adc_baseline
# wfs

In [ ]:
# sys.exit()

In [ ]:
# m.midas

In [ ]:
# m.midas.event

In [ ]:
# non_zero_array_cntr = 0
# for arb_iter in range(10000):
#     try:
#         m.midas.read().__getstate__()
#         if m.midas.read().nsamples:
#             non_zero_array_cntr +=1 
#     except IndexError:
#         continue

# print(f'non_zero_array_cntr: {non_zero_array_cntr}')

In [ ]:
# non_zero_array_cntr = 0
# for arb_iter in range(10000):
#     m.midas.read().__getstate__()
#     if m.midas.read().nsamples:
#         non_zero_array_cntr +=1 

# print(f'non_zero_array_cntr: {non_zero_array_cntr}')

In [ ]:
# m.midas.read().__getstate__()
# m.midas.read().zlecompressed # This shows whether data is zle compressed.
# m.midas.read().nsamples
# m.midas.read().nsamples # number of samples
# m.midas.read().adc_data.shape
### m.midas.read().event_size
# m.midas.read().trigger_time
# m.midas.read().n32samples # not an attribute
# m.midas.nsamples not an attribute

# 4. <a id='toc4_'></a>[Analysis](#toc0_)

## 4.1. <a id='toc4_1_'></a>[data selection](#toc0_)

In [ ]:
event_id = 1000

In [ ]:
wf = wfs[event_id]

In [ ]:
# n_channel = 2
n_channel = 0

In [ ]:
tmin = 0
tmax = min(wf[n_channel].shape[0], 4000)

## 4.2. <a id='toc4_2_'></a>[Plotting data](#toc0_)

In [ ]:
# plt.close(2)
# plt.figure(2)
# plt.title('transformed data')
# # from matplotlib.offsetbox import AnchoredText
# for _c in range(3):
#     plt.plot(wf[_c], label=f'channel {_c}')
# # plt.plot(wf, label=f'channel {_c}')

# plt.legend()
# plt.grid()


In [ ]:
def plooter(x, l):
    plt.figure()
    for ch_id in range(3):
        plt.plot(wfs[x][ch_id], alpha = 1/(1+ 0.5*ch_id),label = f'Channel {ch_id}')
    plt.grid()
    plt.legend()
    plt.title(f'pretrigger sum = {np.sum(wfs[x][ch_id][:400])}')
    if l:
        plt.xlim([0, 500])

In [ ]:
# def str_line(x):
#     return m*x + c

In [ ]:
# wf[2]

In [ ]:
# curve_fit(str_line, np.arange[0,400], wf[2][0:400])

In [ ]:
# histogram of pretrigger window

In [ ]:
wfs.shape

## 4.3. <a id='toc4_3_'></a>[histogramming pretrigger sum values](#toc0_)

In [ ]:
ch_x = 0
pretrigger_sum_ls = []
for event_id in range(wfs.shape[0]):
# for event_id in range(1000):
    # pretrigger_sum_ls.append(np.sum(wfs[event_id][ch_x][:400]))
    pretrigger_sum_ls.append(np.sum(wfs[event_id][ch_x][:350]))

In [ ]:
len(pretrigger_sum_ls)

In [ ]:
plt.figure()
plt.hist(pretrigger_sum_ls, bins = np.arange(-6000, 200_000, 1000));
# plt.hist(pretrigger_sum_ls, bins = np.arange(0, 2_000, 100));
plt.title('Histogram pretrigger sum')
plt.yscale('log')

In [ ]:
sum_arr = np.array(pretrigger_sum_ls)

In [ ]:
# sum_arr[np.where(sum_arr <= 20_000)].shape

In [ ]:
# np.sort(sum_arr[np.where(sum_arr <= 0)])

In [ ]:
# np.sort(sum_arr[np.where(sum_arr <= 3000)])[-100:-10:]

In [ ]:
# sum_arr[np.where(sum_arr <= 3000)].shape

In [ ]:
# l = np.where(sum_arr <= 25000)

In [ ]:
# m = sum_arr[l]

In [ ]:
# m

In [ ]:
# np.sort(m[np.where(m  >= 3000)])[0]

In [ ]:
# np.where(sum_arr <= 6000)

In [ ]:
# sum_arr[np.where(sum_arr <= 3000)].shape

In [ ]:
# sum_arr[np.where(sum_arr <= 6000)].shape

In [ ]:
# np.where(sum_arr == 2913)

In [ ]:
# plooter(4699, 0)

In [ ]:
# plooter(4699, 1)

In [ ]:
# plooter(49971, 0)

In [ ]:
# plooter(49971, 1)

In [ ]:
# plooter(43686, 0)

In [ ]:
# plooter(43686, 1)

In [ ]:
# plooter(25129, 0)

In [ ]:
# np.sum(np.abs(wfs[25129][2][:350]))

In [ ]:
# plooter(25129, 1)

In [ ]:
# plt.figure()
# plt.plot(wfs[25129][0], label='0')
# plt.plot(wfs[25129][1], label='1')
# plt.plot(wfs[25129][2], alpha=0.5, label='2')
# plt.title('Event 25129')
# plt.grid()
# plt.legend()

In [ ]:
# np.sum(wfs[25129][2][:350])

In [ ]:
# plooter(34444, 0)

flagged events
1. 34444
2. 25129

In [ ]:
# plt.figure(figsize=(10, 5))
# # plt.plot(wfs[34444][0], label='0')
# # plt.plot(wfs[34444][1], label='1')
# plt.plot(wfs[34444][2], label='2')
# plt.grid()

In [ ]:
# np.sum(wfs[34444][2])

In [ ]:
# np.sum(wfs[34444][2][:350])

In [ ]:
# plooter(34444, 1)

In [ ]:
# plooter(25522, 0)

In [ ]:
# plooter(25522, 1)

In [ ]:
# plooter(237, 0)

In [ ]:
# plooter(237, 1)

In [ ]:
# plooter(9370, 0)

In [ ]:
# plooter(9370, 1)

In [ ]:
# plooter(39661, 0)

In [ ]:
# plooter(39661, 1)

In [ ]:
# plooter(49316, 0)

In [ ]:
# plooter(49316, 1)

In [ ]:
# plooter(29020, 0)

In [ ]:
# plooter(29020, 1)

In [ ]:
# plooter(35670, 0)

In [ ]:
# plooter(35670, 1)

## 4.4. <a id='toc4_4_'></a>[Repetitive pattern seen in Ch-2 data](#toc0_)

Fourier Transform

In [ ]:
# # event_id = 49998
# event_id = 20_000
# plt.figure()
# for ch in range(3):
#     plt.plot(wfs[event_id][ch], label=f'{ch}')
# plt.legend()
# plt.grid()

### 4.4.1. <a id='toc4_4_1_'></a>[Fourier Transform](#toc0_)

In [ ]:
ft_1 = np.fft.fft(wfs[25129][1])
fig_001, ax_001 = plt.subplots(1, 2, figsize=(10, 5))
ax_001[0].set_title('hist FT of 1')
ax_001[0].hist(ft_1.real, bins=100);
ax_001[1].hist(wfs[25129][1], bins=100);

In [ ]:
ft_2 = np.fft.fft(wfs[25129][2])
fig_002, ax_002 = plt.subplots(1, 2, figsize=(10, 5))
ax_002[0].set_title('hist FT of 2')
ax_002[0].hist(ft_2.real, bins=100);
ax_002[1].hist(wfs[25129][2], bins=100);

## 4.5. <a id='toc4_5_'></a>[simultaneity of pulses](#toc0_)

In [ ]:
plt.close(222)
event_id = 1001
window_range = np.arange(350, 500)
# window_range = np.arange(0, 4000)
plt.figure(222)
plt.plot(window_range, wfs[event_id][0][window_range], label = '0')
plt.plot(window_range, wfs[event_id][1][window_range], label = '1')
plt.plot(window_range, wfs[event_id][2][window_range], alpha=0.2, label = '2')
plt.legend()
plt.grid()

In [ ]:
peaks0 =find_peaks(wfs[event_id][0][window_range])
peaks1 =find_peaks(wfs[event_id][1][window_range])
peaks2 =find_peaks(wfs[event_id][2][window_range])

In [ ]:
plt.figure(222)
plt.scatter(window_range[peaks0[0]], wfs[event_id][0][window_range][peaks0[0]], s=None, marker ='*', color='red')
plt.scatter(window_range[peaks1[0]], wfs[event_id][1][window_range][peaks1[0]], s=None, marker ='*', color='purple')
plt.scatter(window_range[peaks2[0]], wfs[event_id][2][window_range][peaks2[0]], s=None, marker ='*', color='magenta')

In [ ]:
mp0 = np.argmax(wfs[event_id][0][window_range][peaks0[0]])

In [ ]:
window_range[peaks0[0]][mp0]

In [ ]:
wfs[event_id][0][window_range][peaks0[0]][mp0]

In [ ]:
mp1 = np.argmax(wfs[event_id][1][window_range][peaks1[0]])

In [ ]:
window_range[peaks1[0]][mp1]

In [ ]:
wfs[event_id][1][window_range][peaks1[0]][mp1]

In [ ]:
mp2 = np.argmax(wfs[event_id][2][window_range][peaks2[0]])

In [ ]:
window_range[peaks2[0]][mp2]

In [ ]:
wfs[event_id][2][window_range][peaks2[0]][mp2]

In [ ]:
def pulse_difference(event_id):
    window_range = np.arange(350, 500)
    peaks0 =find_peaks(wfs[event_id][0][window_range])
    peaks1 =find_peaks(wfs[event_id][1][window_range])
    peaks2 =find_peaks(wfs[event_id][2][window_range])

    mp0 = np.argmax(wfs[event_id][0][window_range][peaks0[0]])
    mp1 = np.argmax(wfs[event_id][1][window_range][peaks1[0]])
    mp2 = np.argmax(wfs[event_id][2][window_range][peaks2[0]])

    sample_mp0 = wfs[event_id][0][window_range][peaks0[0]][mp0]
    sample_mp1 = window_range[peaks1[0]][mp1]
    sample_mp2 = window_range[peaks2[0]][mp2]

    return abs(sample_mp1 - sample_mp2)

In [ ]:
pretrigger_sum_post_cut_ls = []
for event_id in range(wfs.shape[0]):
    if np.sum(wfs[event_id][ch_x][:350]) <= 20000:
        if pulse_difference(event_id) <= 40:
            pretrigger_sum_post_cut_ls.append(np.sum(wfs[event_id][ch_x][:350]))

In [ ]:
plt.figure()
plt.hist(pretrigger_sum_post_cut_ls, bins = np.arange(-6000, 20000, 200));
# plt.hist(pretrigger_sum_ls, bins = np.arange(0, 2_000, 100));
plt.title('Histogram pretrigger sum post cut')
plt.yscale('log')

In [ ]:
len(pretrigger_sum_post_cut_ls)

## 4.6. <a id='toc4_6_'></a>[Center of Mass for waveforms](#toc0_)

In [ ]:
# event_id = 49998
# event_id = 25129 # 500
# wfs[event_id][2]
event_id = 276

com_array = np.divide(np.sum(np.multiply(wfs[event_id], np.arange(wfs[event_id].shape[1])), axis=1), 
                      np.sum(wfs[event_id], axis=1)
                    )
plt.figure()
plt.plot(wfs[event_id][1], color='orange', alpha=0.2, label=f'{1}')
plt.plot(wfs[event_id][2], color='green', alpha=0.2 , label=f'{2}')
for ch_id in range(3):
    plt.scatter(com_array[ch_id], np.mean(wfs[event_id], axis=1)[ch_id], marker = '*', label = f'com :{ch_id}')
plt.legend()
plt.grid()
plt.title('Center of Mass approach')

In [ ]:
def calculate_com(event_id):
    event_com = np.divide(np.sum(np.multiply(wfs[event_id], np.arange(wfs[event_id].shape[1])), axis=1), 
                      np.sum(wfs[event_id], axis=1)
                )
    return event_com

In [ ]:
com_dict = {0: [],
            1: [],
            2: []
            }
for event_id in range(wfs.shape[0]):
    com_arr = calculate_com(event_id)
    com_dict[0].append(com_arr[0])
    com_dict[1].append(com_arr[1])
    com_dict[2].append(com_arr[2])

In [ ]:
fig_223, ax_223 = plt.subplots( 3, 1, figsize=(10, 8), sharex=True, sharey = True)
for ch_id in range(3):
    ax_223[ch_id].hist(com_dict[ch_id], bins=np.arange(-5_000, 5_000, 25), 
                        color=f'C{ch_id}', label = f'{ch_id}')    
    ax_223[ch_id].legend()
    ax_223[ch_id].grid()
plt.subplots_adjust(wspace=0.025, hspace=0.025)
fig_223.suptitle('hist of Center Of Mass')

In [ ]:
# above 1000 and greater than 1900
# reject if com in three channels are not simultaneous

## 4.7. <a id='toc4_7_'></a>[full waveform sum](#toc0_)

In [ ]:
wf_sum_ls = []
for event_id in range(wfs.shape[0]):
# for event_id in range(1000):
    wf_sum_ls.append(np.sum(wfs[event_id][ch_x]))

In [ ]:
plt.figure()
plt.hist(wf_sum_ls, bins=500, label = 'Channel 0')
plt.legend()
plt.grid()
plt.title(f'hist of waveform sum')
plt.yscale('log')

In [ ]:
wf_sum_post_cut_ls = []
event_list_post_cut = []
com_post_cut_dict = {0: [],
            1: [],
            2: []
            }

com_threshold = 1500

for event_id in range(wfs.shape[0]):
    # if pretrigger_sum_ls[event_id] <= 4000:
    if pretrigger_sum_ls[event_id] <= 20000: # 1st cut: pretrigger sum
        if pulse_difference(event_id) <= 40: # 2nd cut: simultaneity  
            com_arr = calculate_com(event_id)
            if (np.abs(com_arr[0] - com_arr[1]) <= com_threshold) and (np.abs(com_arr[2] - com_arr[1]) <= com_threshold): # 3rd cut: concurrence of COM
                event_list_post_cut.append(event_id)
                wf_sum_post_cut_ls.append(np.sum(wfs[event_id][ch_x]))
                com_post_cut_dict[0].append(com_arr[0])
                com_post_cut_dict[1].append(com_arr[1])
                com_post_cut_dict[2].append(com_arr[2])

In [ ]:
fig_224, ax_224 = plt.subplots( 3, 1, figsize=(10, 8), sharex=True, sharey = True)
for ch_id in range(3):
    ax_224[ch_id].hist(com_post_cut_dict[ch_id], bins=np.arange(-5_000, 5_000, 25), 
                        color=f'C{ch_id}', label = f'{ch_id}')    
    ax_224[ch_id].legend()
    ax_224[ch_id].grid()
plt.subplots_adjust(wspace=0.025, hspace=0.025)
fig_224.suptitle('hist of Center Of Mass post cuts')

In [ ]:
len(com_post_cut_dict[0])

In [ ]:
plt.figure()
plt.hist(wf_sum_post_cut_ls, bins=500, label = 'Channel 0')
plt.legend()
plt.grid()
plt.title(f'hist of waveform sum post 3 cuts')
plt.yscale('log')

In [ ]:
len(wf_sum_post_cut_ls)

In [ ]:
500 in event_list_post_cut

In [ ]:
34444 in event_list_post_cut

In [ ]:
25129 in event_list_post_cut

In [ ]:
event_list_post_cut[-1::-1]

In [ ]:
50000 in event_list_post_cut

In [ ]:
len(wf_sum_post_cut_ls)

In [ ]:
bad_event_ls = [49971, 43686, 25129, 34444, 25522, 39661, 49316, 29020, 35670, 35670, 49998]

In [ ]:
int_index = 0
for be in bad_event_ls:
    if  be in event_list_post_cut:
        int_index += 1
        print(f'Bad Event Count:{int_index}. Bad event id: {be}')

In [ ]:
perf_counter() - t0

In [ ]:
sys.exit()

# 5. <a id='toc5_'></a>[Exploration](#toc0_)

In [ ]:
wfs.shape

In [ ]:
event_id = 45000

In [ ]:
# for i in range(100_000):
event_id += 1
plooter(event_id, 0)

In [ ]:
event_id

# 6. <a id='toc6_'></a>[Memory diagnostics](#toc0_)

In [ ]:
# ## see memory usage
!cat /proc/meminfo

In [ ]:
# event_catalogue.info(memory_usage='deep')

In [ ]:
# del event_catalogue

In [ ]:
# !cat /proc/meminfo

In [ ]:
# import gc
# gc.isenabled()

In [ ]:
# gc.collect()